# Import Libraries

In [3]:
import requests
import pandas as pd
import time

from datetime import date, datetime, timedelta


In [4]:
%pip install Pulldata==1.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import Pulldata
from Pulldata import myfunctions

I make a "Pulldata" library to include the function of pulling data from Polygon. Inside the "Pulldata", it provides two function: pull_1m_data and pull_6m_data, which respectively pull data every 1 min or 6 min. 

# Process Raw Data

In [77]:
# This function processes the raw data pulled from the Polygon

def ten_hr_table_6m(symbol):
  day = date(year = 2022, month = 11, day =17)
  bars = []
  days_of_data = 1

  for i in range(days_of_data):
      bars += myfunctions.pull_6m_data(symbol, day)
      day -= timedelta(days = 1)
      time.sleep(15)

  symbol = pd.DataFrame(bars)
  symbol["date"] = pd.to_datetime(symbol["t"], unit = "ms")
  symbol =  symbol[["date","h","l","vw"]]
  symbol.columns = ["time","high","low","avg"]
  symbol["vol"] = symbol["high"] - symbol["low"]
  symbol = symbol.sort_values("time")

  count = 0
  symbol.insert(5,"N","")
  symbol["N"][0] = '0'
  while count < 99:
    symbol["N"][count+1] = int(abs((symbol["avg"][count+1]-symbol["avg"][count]) // (0.025*symbol["vol"][count+1])))
    count +=1

  symbol["N"] = symbol["N"].astype(float)
  symbol["FD"] = symbol["N"]/symbol['vol']

  return symbol

# Saving the different currency pairs' data table

In [126]:
currency_pairs = ["USDAUD", "EURGBP","CADUSD","JPYUSD","MXNUSD","USDEUR","CNYUSD","CZKUSD","PLNUSD","INRUSD"]


In [117]:
from sqlalchemy import create_engine
engine = create_engine('sqlite+pysqlite:///hm2.db', echo=False)

In [127]:
for cur in currency_pairs:
  cur_name = cur
  cur = ten_hr_table_6m(cur)
  cur.to_sql(cur_name, con=engine)

In [131]:
for cur in currency_pairs:
  cur_name = cur
  cur = ten_hr_table_6m(cur)
  cur.to_csv(cur_name+'.csv')